In [2]:
import seaborn as sns
import os
from scipy.stats import kurtosis, skew
import pickle

from rcv_distribution import *
from MDS_analysis import *
from voting_rules import *
import random
from itertools import permutations
from collections import defaultdict
from collections import Counter

c:\Users\mahsh\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\mahsh\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
c:\Users\mahsh\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
directory = "dataverse_files"
table = pd.read_csv("diff.csv")
for filename in os.listdir(directory):
    try:
        csv = os.path.join(directory, filename)
        ballots, candidates = parse_election_data(csv)

        # Perform the RCV analysis
        test = perform_rcv_analysis(ballots, candidates, n_init = 100, max_itr = 1000, n_runs=1000, metric=False)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances_original = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)

        consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances_original)
        table.loc[table["filename"]==filename, "gamma_n_init"] = gamma

        plot_KDE(ballots, normalized_distances_original, filename, ignore=False, save=True)

        freqs = get_frequency(ballots, candidates)
        ignore = [] 
        for c in freqs:
            if freqs[c] < 0.02:
                ignore.append(c)
        if len(ignore) > 0:
            ballots, candidates = parse_election_data(os.path.join("dataverse_files", filename), ignore_values=ignore)
            # Perform the RCV analysis
            test = perform_rcv_analysis(ballots, candidates, n_init = 100, max_itr = 1000, n_runs=1000, metric=False)
            mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

            # Get the normalized distances between candidates and plot the MDS analysis
            normalized_distances_ignored = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)

            consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances_ignored)

            table.loc[table["filename"]==filename, "gamma_ignored"] = gamma

            plot_KDE(ballots, normalized_distances_original, filename, ignore=True, save=True)
    
    except Exception as e:
        # Print the summary of the error
        print(f"An error occurred: {e}")

table.to_csv("diff2.csv")


In [5]:
def save_data(filename, data_dict, data_list, save_folder):
    # Extract the base filename without extension
    base_filename = os.path.splitext(os.path.basename(filename))[0]

    # Create the save paths for dictionary and list
    dict_save_path = os.path.join(save_folder, f"{base_filename}_ballots.pkl")
    list_save_path = os.path.join(save_folder, f"{base_filename}_candidates.pkl")

    # Save the dictionary
    with open(dict_save_path, 'wb') as dict_file:
        pickle.dump(data_dict, dict_file)

    # Save the list
    with open(list_save_path, 'wb') as list_file:
        pickle.dump(data_list, list_file)

In [3]:
def load_data(filename, save_folder):
    # Extract the base filename without extension
    base_filename = os.path.splitext(os.path.basename(filename))[0]

    # Create the save paths for dictionary and list
    dict_load_path = os.path.join(save_folder, f"{base_filename}_dict.pkl")
    list_load_path = os.path.join(save_folder, f"{base_filename}_list.pkl")

    # Load the dictionary
    with open(dict_load_path, 'rb') as dict_file:
        data_dict = pickle.load(dict_file)

    # Load the list
    with open(list_load_path, 'rb') as list_file:
        data_list = pickle.load(list_file)

    return data_dict, data_list



In [8]:
directory = "dataverse_files"
table = pd.read_csv("diff.csv")
for file in os.listdir(directory):
    try:
        csv = os.path.join(directory, file)
        ballots, candidates = parse_election_data(csv)
        filename = file[0:-4]
        save_data(filename, ballots, candidates, "saved_ballots_and_candidates")

        candidate_positions_file = pd.read_csv("null_elections/" + filename + ".csv")
        normalized_positions = pd.Series(candidate_positions_file['position'].values, index=candidate_positions_file['candidate']).to_dict()

        plot_KDE(ballots, normalized_positions, file, ignore=False, save=True, directory="KDE_new")

    except Exception as e:
        # Print the summary of the error
        print(file, " ", e)

table.to_csv("diff2.csv")


KeyboardInterrupt: 